In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from scipy.ndimage import gaussian_filter
from sklearn.preprocessing import OneHotEncoder
import random
from keras.utils.np_utils import to_categorical   
from tensorflow.keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import class_weight

In [3]:
train_list_x = os.listdir("../input/mlsp-train2/X")

In [4]:
x_train = []
y_train = []
for file in train_list_x :
    x_train.append(np.load(os.path.join("../input/mlsp-train2/X" , file) ,allow_pickle = True))
    y_train.append(np.load(os.path.join("../input/mlsp-train2/Y" , "eventroll"+ file[7:]) ,allow_pickle = True))
x_train= np.array(x_train)
y_train = np.array(y_train)

In [5]:
np.save("./x_train.npy" , x_train)
np.save("./y_train.npy" , y_train)

In [6]:
x_train = np.load("./x_train.npy")
y_train = np.load("./y_train.npy")

In [7]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.delete(active_events, 8)

In [8]:
y = []
for i in y_train :
    y.append(eventroll_to_multihot_vector(i))

In [9]:
y_train = np.array(y)

In [10]:
y_train.sum(axis = 0)

array([1499.,  993., 1128., 2440., 1412., 1096., 1349., 1219., 9201.,
       1046.])

**Clearly there is heavy data imbalance**

In [11]:
def addDim(mat) :
  return np.expand_dims(mat , axis = 2)
def reduceDim(mat) :
  return mat[0]

In [12]:
def addDim1(mat) :
  return np.expand_dims(mat , axis = 3)


In [13]:
x_binary = []
y_binary = []
count = 0
for i in range(10000) :
    if y_train[i][8] == 0 :
        x_binary.append(reduceDim(addDim1(x_train[i])))
        y_binary.append([0,1])
    elif count<800 :
        x_binary.append(reduceDim(addDim1(x_train[i])))
        y_binary.append([1,0])
        count += 1
x_binary = np.array(x_binary)
y_binary = np.array(y_binary)

In [14]:
np.shape(x_binary)

(1599, 64, 1000, 1)

In [104]:
def cnn1() :
    cnn = Sequential()
    cnn.add(layers.Conv2D(16,(3,3),(2,2), padding = 'same' , activation = 'relu' ,input_shape = (64,1000,1,)))
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.MaxPool2D())
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.Conv2D(32 , (3,3) ,padding = 'same' , activation = 'relu'))
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.MaxPool2D())
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.Conv2D(64 , (3,3) ,(2,2), padding = 'same' , activation = 'relu'))
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.Conv2D(64 , (3,3) , padding = 'same'))
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.GlobalAveragePooling2D())
    cnn.add(layers.Dense(32 , activation = 'relu'))
    cnn.add(layers.Dense(2 , activation = 'softmax'))
    cnn.summary()
    return cnn

In [105]:
cnn_model = cnn1()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 32, 500, 16)       160       
_________________________________________________________________
batch_normalization_30 (Batc (None, 32, 500, 16)       64        
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 250, 16)       0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 16, 250, 16)       64        
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 250, 32)       4640      
_________________________________________________________________
batch_normalization_32 (Batc (None, 16, 250, 32)       128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 125, 32)       

In [106]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [107]:
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss= 'categorical_crossentropy' , metrics=['accuracy'])

In [124]:
history_cnn_model1 = cnn_model.fit(
    x = x_binary , y = y_binary,
    validation_split = 0.2,
    epochs = 25 , 
    steps_per_epoch = 50 ,
    batch_size = 32,
)

Epoch 1/25
50/50 [==============================] - 10s 183ms/step - loss: 0.5789 - accuracy: 0.8512 - val_loss: 2.3202 - val_accuracy: 0.9190
Epoch 2/25
50/50 [==============================] - 10s 194ms/step - loss: 0.2285 - accuracy: 0.9169 - val_loss: 0.6400 - val_accuracy: 0.9190
Epoch 3/25
50/50 [==============================] - 9s 175ms/step - loss: 0.1733 - accuracy: 0.9256 - val_loss: 0.6799 - val_accuracy: 0.9190
Epoch 4/25
50/50 [==============================] - 9s 176ms/step - loss: 0.1747 - accuracy: 0.9331 - val_loss: 0.4358 - val_accuracy: 0.9190
Epoch 5/25
50/50 [==============================] - 10s 191ms/step - loss: 0.1821 - accuracy: 0.9231 - val_loss: 0.3549 - val_accuracy: 0.9190
Epoch 6/25
50/50 [==============================] - 9s 175ms/step - loss: 0.1572 - accuracy: 0.9306 - val_loss: 0.2291 - val_accuracy: 0.9170
Epoch 7/25
50/50 [==============================] - 9s 174ms/step - loss: 0.1458 - accuracy: 0.9425 - val_loss: 0.2228 - val_accuracy: 0.9200
Epo

In [126]:
cnn_model.save("./binary3.h5")

In [60]:
x_binary_test = []
y_binary_test = []

for i in range(10000) :
    if y_train[i][8] == 0 :
        x_binary_test.append(reduceDim(addDim1(x_train[i])))
        y_binary_test.append([0,1])
    else :
        x_binary_test.append(reduceDim(addDim1(x_train[i])))
        y_binary_test.append([1,0])
        count += 1
x_binary_test = np.array(x_binary_test)
y_binary_test = np.array(y_binary_test)

In [79]:
speech_pred = cnn_model.predict(x = x_binary_test)

In [81]:
speech_pred = np.where(speech_pred >=0.5 ,1 ,0)

In [125]:
print(y_binary_test.sum(axis =0))
print(speech_pred.sum(axis=0))

[1246  754]
[8876 1124]


In [82]:
from sklearn.metrics import f1_score,precision_score,recall_score

print(precision_score(y_binary_test, speech_pred, average='samples'))
print(recall_score(y_binary_test, speech_pred , average='samples'))
print(f1_score(y_binary_test, speech_pred, average='samples'))

0.6265
0.6265
0.6265


In [15]:
y_multihot = []
for labels in y_train :
    y_multihot.append(np.delete(labels,8))
y_multihot = np.array(y_multihot)

In [16]:
y_multihot.sum(axis = 0)

array([1499.,  993., 1128., 2440., 1412., 1096., 1349., 1219., 1046.])

In [17]:
x_data_reduced = []
for x in x_train :
    x_data_reduced.append(addDim(reduceDim(x)))
x_data_reduced = np.array(x_data_reduced)

In [18]:
def cnn2() :
    cnn = Sequential()
    cnn.add(layers.Conv2D(16,(3,3),(2,2), padding = 'same' , activation = 'relu' ,input_shape = (64,1000,1,)))
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.MaxPool2D())
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.Conv2D(32 , (3,3) ,padding = 'same' , activation = 'relu'))
    cnn.add(layers.MaxPool2D())
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.Conv2D(64 , (3,3) ,(2,2), padding = 'same' , activation = 'relu'))
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.Conv2D(64 , (3,3) , padding = 'same'))
    cnn.add(layers.BatchNormalization())
    cnn.add(layers.GlobalAveragePooling2D())
    cnn.add(layers.Dense(64 , activation = 'relu'))
    cnn.add(layers.Dense(9 , activation = 'sigmoid'))
    cnn.summary()
    return cnn

In [19]:
cnn_model2 = cnn2()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 500, 16)       160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 250, 16)       2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 125, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 125, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 125, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 125, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 62, 32)         0

2022-11-14 04:46:00.789554: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [20]:
cnn_model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss= 'binary_crossentropy' , metrics=['binary_accuracy'])

In [21]:
np.shape(x_data_reduced)

(10000, 64, 1000, 1)

In [22]:
np.shape(y_multihot)

(10000, 9)

In [23]:
callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0.0005,
    patience= 25,restore_best_weights=True)

In [24]:
history_cnn_model2 = cnn_model2.fit(
    x = x_data_reduced , y = y_multihot,
    validation_split = 0.2,
    epochs = 80 , 
    steps_per_epoch = 100 ,
    batch_size = 32,
    
#     callbacks = callback
)

2022-11-14 04:46:05.200079: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/80
100/100 [==============================] - 13s 111ms/step - loss: 0.3851 - binary_accuracy: 0.8510 - val_loss: 0.4367 - val_binary_accuracy: 0.8667
Epoch 2/80
100/100 [==============================] - 11s 106ms/step - loss: 0.3173 - binary_accuracy: 0.8750 - val_loss: 0.4570 - val_binary_accuracy: 0.8667
Epoch 3/80
100/100 [==============================] - 10s 97ms/step - loss: 0.2989 - binary_accuracy: 0.8812 - val_loss: 0.4832 - val_binary_accuracy: 0.8415
Epoch 4/80
100/100 [==============================] - 10s 96ms/step - loss: 0.2884 - binary_accuracy: 0.8834 - val_loss: 0.5042 - val_binary_accuracy: 0.8626
Epoch 5/80
100/100 [==============================] - 10s 103ms/step - loss: 0.2699 - binary_accuracy: 0.8913 - val_loss: 0.4807 - val_binary_accuracy: 0.8575
Epoch 6/80
100/100 [==============================] - 9s 94ms/step - loss: 0.2632 - binary_accuracy: 0.8942 - val_loss: 0.6128 - val_binary_accuracy: 0.7567
Epoch 7/80
100/100 [==============================

In [25]:
pred = cnn_model2.predict(x = x_data_reduced)

In [26]:
pred = np.where(pred>=0.5 , 1 , 0)

In [27]:
print(pred.sum(axis = 0))
print(y_multihot.sum(axis=0))

[1556  833  827 2728 1023  968 1419  967 1136]
[1499.  993. 1128. 2440. 1412. 1096. 1349. 1219. 1046.]


In [28]:
cnn_model2.save("./multi6.h5")

# **Validation**

In [ ]:
cnn_model2 = keras.models.load_model("./multi5.h5")

In [29]:
test_list_x = os.listdir("../input/mlsp-assignment2-val/dataset/X")
x_test = []
y_test = []
for file in test_list_x :
    x_test.append(np.load(os.path.join("../input/mlsp-assignment2-val/dataset/X" , file) ,allow_pickle = True))
    y_test.append(np.load(os.path.join("../input/mlsp-assignment2-val/dataset/Y" , "eventroll"+ file[7:]) ,allow_pickle = True))
x_test= np.array(x_test)
y1 = []
for i in y_test :
    y1.append(eventroll_to_multihot_vector(i))
y_test = np.array(y1)
np.save("./x_val.npy" , x_test)
np.save("./y_val.npy",y_test)
print(y_test.sum(axis = 0))

[ 392.  436.  274.  444.  306.  221.  130.  143. 1246.  151.]


In [30]:
x_test = np.load("./x_val.npy")
y_test = np.load("./y_val.npy")

In [31]:
x_binary_test = []
y_binary_test = []

for i in range(2000) :
    if y_test[i][8] == 0 :
        x_binary_test.append(reduceDim(addDim1(x_test[i])))
        y_binary_test.append([0,1])
    else :
        x_binary_test.append(reduceDim(addDim1(x_test[i])))
        y_binary_test.append([1,0])
        count += 1
x_binary_test = np.array(x_binary_test)
y_binary_test = np.array(y_binary_test)

In [34]:
pred_speech_val =cnn_model.predict(x = x_binary_test)
pred_speech_val = np.where(pred_speech_val>=0.5 , 1, 0)
print(pred_speech_val.sum(axis=0))
print(y_binary_test.sum(axis = 0))

[1993    7]
[1246  754]


In [35]:
y_multihot_val = []
for labels in y_test :
    y_multihot_val.append(np.delete(labels,8))
y_multihot_val = np.array(y_multihot_val)

In [36]:
y_multihot_val.sum(axis = 0)

array([392., 436., 274., 444., 306., 221., 130., 143., 151.])

In [37]:
x_data_reduced_val = []
for x in x_test :
    x_data_reduced_val.append(addDim(reduceDim(x)))
x_data_reduced_val = np.array(x_data_reduced_val)

In [38]:
pred_val = cnn_model2.predict(x = x_data_reduced_val)

In [39]:
pred_val = np.where(pred_val>=0.5 , 1,0)
pred_val.sum(axis = 0)

array([416, 406, 177, 427, 190, 131,  90,  73, 119])

In [40]:
from sklearn.metrics import f1_score,precision_score,recall_score

print(precision_score(y_multihot_val, pred_val, average='samples'))
print(recall_score(y_multihot_val, pred_val , average='samples'))
print(f1_score(y_multihot_val, pred_val, average='samples'))

0.74775
0.6922083333333334
0.7017666666666666


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


# **Testing**

In [94]:
model = keras.models.load_model("./multi3.h5")
dict = {}
test_list = os.listdir("../input/mlsp-test/test/X")
for file in test_list:
    a = np.load(os.path.join("../input/mlsp-test/test/X", file))
    a =  np.expand_dims(a, axis = 3)
    dict[file] = model.predict(a)
pred = np.array(list(dict.values()))
pred = np.where(pred>=0.5 , 1, 0)
print(pred.sum(axis =0))

[[408 131 176 705 235 203 358 236 299]]


In [95]:
print(np.shape(pred))

(2500, 1, 9)


In [98]:
t = []
for p in pred :
    t.append(p[0])
pred = np.array(t)

In [96]:
model2 = keras.models.load_model("./binary2.h5")
dict1 = {}
test_list = os.listdir("../input/mlsp-test/test/X")
for file in test_list:
    a = np.load(os.path.join("../input/mlsp-test/test/X", file))
    a =  np.expand_dims(a, axis = 3)
    dict1[file] = model2.predict(a)
pred1 = np.array(list(dict1.values()))
pred1 = np.where(pred1>=0.5 , 1, 0)
print(pred1.sum(axis =0))

[[2490   10]]


In [100]:
print(np.shape(pred1))

(2500, 1, 2)


In [101]:
t = []
for p in pred1 :
    t.append(p[0])
pred1 = np.array(t)

In [102]:
final_pred = []
for i in range(2500) :
    final_pred.append(np.insert(pred[i] , 8 , pred1[i][0]))
pred = np.array(final_pred)

In [103]:
print(np.shape(pred))

(2500, 10)


In [104]:
print(np.shape(pred1[0]))
print(pred1[0][0])

(2,)
1


In [106]:
results = []
for i in pred :
    a = []
    for j in range(10) :
        a.append(i[j])
    results.append(a)

In [71]:
field_ids = list(dict.keys())

In [72]:
names = {
    
    0: 'Alarm_bell_ringing', 
    1: 'Blender', 
    2: 'Cat', 
    3: 'Dishes', 
    4: 'Dog',
    5: 'Electric_shaver_toothbrush', 
    6: 'Frying', 
    7: 'Running_water',
    8: 'Speech', 
    9: 'Vacuum_cleaner'


}

In [73]:
arr = []
for t in results :
    vec = []
    for i in range(10) :
        if t[i]==1:
            vec.append(names[i])
    arr.append(vec)

In [75]:
df = pd.DataFrame(list(zip(field_ids, arr)),columns =['fileid', 'prediction'])

In [108]:
df.to_csv("./pred.csv")
